In [18]:
import os
import networkx as nx
import matplotlib.pyplot as plt
import cython
import numpy as np
import pandas as pd
import datetime
import pylab

In [19]:
# load in merged data
data_path = '../../../UCB_dept_merge_CorrectDeptLabels.csv'
merged = pd.read_csv(data_path)

In [20]:
#clean up the department dataset
def convert_strings_to_specials(s):
    s = s.replace(' ', '_')
    s = s.replace(':', '_')
    s = s.replace('#', 'num')
    s = s.lower()
    return s
merged.columns = [convert_strings_to_specials(col) for col in merged.columns]

In [21]:
#convert creation_date to datetime object
merged['creation_date']=pd.to_datetime(pd.Series(merged['creation_date']))

In [31]:
month = [date.month for date in merged['creation_date']]
year = [date.year for date in merged['creation_date']]
day = [date.day for date in merged['creation_date']]
merged['month'] = month
merged['year'] = year
merged['day'] = day

In [32]:
merged.head()

,unnamed__0,po_id,po_num,creation_date,supplier_name,item_type,product_description,manufacturer,quantity,unit_price,...,buyer__last_name,po_closed_date,department_name,spend,department_number,department_name_stripped,department_name_update,month,year,day
0,0,29847876,BB00195887,2013-05-31,GIVE SOMETHING BACK,SQ Hosted Product,"PORTABLE COMBINATION LAPTOP LOCK, 6 FT. CARBON...",KENSINGTON,30,24.32,...,Miller,2013-07-16 00:00:00,UIAPA UB Academic Year,795.26,UIAPA,UB Academic Year,NaN,5,2013,31
1,1,29847864,BB00195886,2013-05-31,GRAINGER INC,PunchOut Product,"Wall Mount Fan, Oscillating, Number of Speeds ...",AIR KING,1,35.58,...,Pinkston,2013-08-08 00:00:00,UKHDS Unit 1 Apt Admin,38.78,UKHDS,Unit 1 Apt Admin,Housing & Dining Services,5,2013,31
2,2,29847796,BB00195884,2013-05-31,BELLCO GLASS INC,NonCatalog Product,"Septum Stopper, 20mm Blue Butyl Rubber QtyPerC...",NaN,1,77.41,...,Wolf,NaN,CCHEM RES Research,104.67,CCHEM,RES Research,Dept Of Chemistry,5,2013,31
3,3,29847820,BB00195885,2013-05-31,CHEMGLASS LIFE SCIENCES LLC,NonCatalog Product,"Column, Chromatography, 24/40 Outer Joint, 100...",NaN,1,108.00,...,Wolf,2014-10-30 00:00:00,CCHEM RES Research,71.77,CCHEM,RES Research,Dept Of Chemistry,5,2013,31
4,4,29847565,BB00195881,2013-05-31,FISHER SCIENTIFIC,SQ Hosted Product,"Bottles, Media/Lab; Wheaton;Graduated; With ru...",Wheaton Science Products Inc,1,135.38,...,RIO,NaN,IMMCB BH Research,317.31,IMMCB,BH Research,Molecular & Cell Biology,5,2013,31


In [35]:
#iterator for getting month,year
def month_year_iter( start_month, start_year, end_month, end_year ):
    ym_start= 12*start_year + start_month - 1
    ym_end= 12*end_year + end_month - 1
    for ym in range( ym_start, ym_end ):
        y, m = divmod( ym, 12 )
        yield y, m+1

In [63]:
#function to give a bitartite graph and centrality measure of department and supplier by time
def testCentralityGraph(data, set1, set2, time_column, month_column, year_column):
    results = []
    def calculate(set1, set2, current):
        G = nx.Graph()
        G.add_nodes_from(current[set1],bipartite = 0)
        G.add_nodes_from(current[set2],bipartite = 1)
        edgeList = [tuple(x) for x in current.values]
        G.add_edges_from(edgeList)
        pos = nx.networkx.spring_layout(G)
        nx.draw_networkx_nodes(G, pos, nodelist = current[set1].tolist(), node_color = 'r', node_size = 50)
        nx.draw_networkx_nodes(G, pos, nodelist = current[set2].tolist(), node_color= 'w', node_size=50)
        #draw edges
        nx.draw_networkx_edges(G,pos,width=0.5,alpha=0.5)
        nx.draw_networkx_edges(G,pos,edgelist=edgeList)
        bottom_nodes, top_nodes = nx.bipartite.sets(G)
        projected_bottom = nx.bipartite.projected_graph(G, bottom_nodes, multigraph = True)
        projected_top = nx.bipartite.projected_graph(G, top_nodes, multigraph = True)
        top_density, bottom_density = nx.bipartite.density(G, top_nodes), nx.bipartite.density(G, bottom_nodes)
        top_degree = nx.degree_centrality(projected_top)
        bot_degree = nx.degree_centrality(projected_bottom)
        top_between = nx.betweenness_centrality(projected_top)
        bot_between = nx.betweenness_centrality(projected_bottom)
        top_projected_density = nx.density(projected_top)
        bot_projected_density = nx.density(projected_bottom)
        
        return (G,projected_top,projected_bottom,top_density,bottom_density,top_degree,bot_degree,top_between,bot_between,
                top_projected_density,bot_projected_density)
            
    firstDate = min(data[time_column])
    lastDate = max(data[time_column])
    iter = month_year_iter(firstDate.month, firstDate.year, lastDate.month, lastDate.year)
    for month,year in iter:
        current = data[data[month_column] == month]
        current = data[data[year_column] == year]
        current = current[[set1,set2]]
        current.dropna()
        temp = calculate(set1,set2,current)
        results +=[(temp,month,year)]

    return results

In [64]:
masterlist = testCentralityGraph(merged,'department_name_update', 'supplier_name','creation_date','month','year')

In [56]:
#subset by date
first3Months = deptSupplier[deptSupplier['creation_date'] < datetime.datetime(2012,3,1)]
first3Months = first3Months.dropna() 
len(first3Months)

636

In [66]:
#keep only the unique values
wanted = first3Months[['department_name','supplier_name']]
wanted.drop_duplicates()

NameError: name 'first3Months' is not defined

In [58]:
#make a graph
test= nx.Graph()
edges = [tuple(x) for x in wanted.values]
test.add_edges_from(edges)
#position = {department:0 for department in wanted["department_name"]}
#for supplier in wanted["supplier_name"]: position[supplier] = 1
values = range(0,len(test.nodes()))
#len(position) ==len(values)

In [ ]:
nx.draw(test, node_color = values,with_labels = True)
plt.show()
plt.savefig("test.png")